In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm

**If using Stargazer to export LaTeX code:** \
!pip install Stargazer \
from stargazer.stargazer import Stargazer

In [2]:
data = pd.read_csv('factordata.csv')

# Multiplying by Indicators
data['IndGMB_U'] = data['GMB_U'] * data['Method_Estimated']
data['IndGMB_S'] = data['GMB_S'] * data['Method_Estimated']
data['IndMkt-RF'] = data['Mkt-RF'] * data['Method_Estimated']
data['IndSMB'] = data['SMB'] * data['Method_Estimated']
data['IndHML'] = data['HML'] * data['Method_Estimated']
data['IndRMW'] = data['RMW'] * data['Method_Estimated']
data['IndCMA'] = data['CMA'] * data['Method_Estimated']

print(data)

           cusip  year  Month  monthyear  Mkt-RF   SMB   HML    RF   RMW  \
0       46603210  2015      1     201501   -3.11 -0.92 -3.59  0.00  1.61   
1       46603210  2015      2     201502    6.13  0.32 -1.86  0.00 -1.12   
2       46603210  2015      3     201503   -1.12  3.07 -0.38  0.00  0.09   
3       46603210  2015      4     201504    0.59 -3.09  1.82  0.00  0.06   
4       46603210  2015      5     201505    1.36  0.84 -1.15  0.00 -1.80   
...          ...   ...    ...        ...     ...   ...   ...   ...   ...   
295824  12503M10  2023      8     202308   -2.39 -3.68 -1.08  0.45  3.42   
295825  12503M10  2023      9     202309   -5.24 -1.79  1.45  0.43  1.85   
295826  12503M10  2023     10     202310   -3.18 -4.05  0.19  0.47  2.47   
295827  12503M10  2023     11     202311    8.83 -0.11  1.66  0.44 -3.81   
295828  12503M10  2023     12     202312    4.87  7.33  4.92  0.43 -3.04   

         CMA  ...  Method_Estimated     GMB_U     GMB_S  IndGMB_U  IndGMB_S  \
0      -

In [3]:
# Get sector dummies!
sectordummies = pd.get_dummies(data['sector'], dtype = int, prefix='sector')
# Concatenate dummy variables with main DataFrame
withsectors = pd.concat([data, sectordummies], axis=1)

# Multiplying by Indicators (REMOVING RETAIL (median))
withsectors['IndU_Acc'] = withsectors['GMB_U'] * withsectors['sector_AccomFood']
withsectors['IndU_Cons'] = withsectors['GMB_U'] * withsectors['sector_Construction']
withsectors['IndU_Fin'] = withsectors['GMB_U'] * withsectors['sector_FinanceIns']
withsectors['IndU_HC'] = withsectors['GMB_U'] * withsectors['sector_Healthcare']
withsectors['IndU_Info'] = withsectors['GMB_U'] * withsectors['sector_Information']
withsectors['IndU_Man'] = withsectors['GMB_U'] * withsectors['sector_Manufacturing']
withsectors['IndU_Oil'] = withsectors['GMB_U'] * withsectors['sector_OilGas']
withsectors['IndU_RE'] = withsectors['GMB_U'] * withsectors['sector_RealEstate']
withsectors['IndU_Serv'] = withsectors['GMB_U'] * withsectors['sector_Services']
withsectors['IndU_Transp'] = withsectors['GMB_U'] * withsectors['sector_TransportWarehouse']
withsectors['IndU_Util'] = withsectors['GMB_U'] * withsectors['sector_Utilities']
withsectors['IndU_Whol'] = withsectors['GMB_U'] * withsectors['sector_Wholesale']

withsectors['IndS_Acc'] = withsectors['GMB_S'] * withsectors['sector_AccomFood']
withsectors['IndS_Cons'] = withsectors['GMB_S'] * withsectors['sector_Construction']
withsectors['IndS_Fin'] = withsectors['GMB_S'] * withsectors['sector_FinanceIns']
withsectors['IndS_HC'] = withsectors['GMB_S'] * withsectors['sector_Healthcare']
withsectors['IndS_Info'] = withsectors['GMB_S'] * withsectors['sector_Information']
withsectors['IndS_Man'] = withsectors['GMB_S'] * withsectors['sector_Manufacturing']
withsectors['IndS_Oil'] = withsectors['GMB_S'] * withsectors['sector_OilGas']
withsectors['IndS_RE'] = withsectors['GMB_S'] * withsectors['sector_RealEstate']
withsectors['IndS_Serv'] = withsectors['GMB_S'] * withsectors['sector_Services']
withsectors['IndS_Transp'] = withsectors['GMB_S'] * withsectors['sector_TransportWarehouse']
withsectors['IndS_Util'] = withsectors['GMB_S'] * withsectors['sector_Utilities']
withsectors['IndS_Whol'] = withsectors['GMB_S'] * withsectors['sector_Wholesale']

In [4]:
# Get time dummies!
yeardummies = pd.get_dummies(data['year'], dtype = int, prefix='year')
# Drop first year (2002) to avoid multicollinearity with time dummies
yeardummies.drop('year_2002', axis=1, inplace=True)
# Concatenate dummy variables with main DataFrame
factordata = pd.concat([withsectors, yeardummies], axis=1)

# Table 1: Baseline

**Just the FF3**

In [5]:
T1X1 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y1 = factordata["ExcessReturn"]
T1model1 = sm.OLS(T1y1, T1X1).fit(cov_type='HC1')
print(T1model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                 1.643e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:04   Log-Likelihood:             1.7631e+05
No. Observations:              295829   AIC:                        -3.526e+05
Df Residuals:                  295804   BIC:                        -3.523e+05
Df Model:                          24                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1278      0.002    -62.963      0.0

**Just the FF5**

In [6]:
T1X2 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y2 = factordata["ExcessReturn"]
T1model2 = sm.OLS(T1y2, T1X2).fit(cov_type='HC1')
print(T1model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                 1.528e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:04   Log-Likelihood:             1.7658e+05
No. Observations:              295829   AIC:                        -3.531e+05
Df Residuals:                  295802   BIC:                        -3.528e+05
Df Model:                          26                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1235      0.002    -59.131      0.0

**FF3 + GMB_U**

In [7]:
T1X3 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y3 = factordata["ExcessReturn"]
T1model3 = sm.OLS(T1y3, T1X3).fit(cov_type='HC1')
print(T1model3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                 1.575e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:04   Log-Likelihood:             1.7715e+05
No. Observations:              295829   AIC:                        -3.542e+05
Df Residuals:                  295803   BIC:                        -3.540e+05
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1282      0.002    -62.950      0.0

**FF5 + GMB_U**

In [8]:
T1X4 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y4 = factordata["ExcessReturn"]
T1model4 = sm.OLS(T1y4, T1X4).fit(cov_type='HC1')
print(T1model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.468e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:05   Log-Likelihood:             1.7776e+05
No. Observations:              295829   AIC:                        -3.555e+05
Df Residuals:                  295801   BIC:                        -3.552e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1168      0.002    -55.273      0.0

**FF3 + GMB_S**

In [9]:
T1X5 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y5 = factordata["ExcessReturn"]
T1model5 = sm.OLS(T1y5, T1X5).fit(cov_type='HC1')
print(T1model5.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                 1.582e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:05   Log-Likelihood:             1.7709e+05
No. Observations:              295829   AIC:                        -3.541e+05
Df Residuals:                  295803   BIC:                        -3.539e+05
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1293      0.002    -63.718      0.0

**FF5 + GMB_S**

In [10]:
T1X6 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA','GMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y6 = factordata["ExcessReturn"]
T1model6 = sm.OLS(T1y6, T1X6).fit(cov_type='HC1')
print(T1model6.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.472e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:             1.7776e+05
No. Observations:              295829   AIC:                        -3.555e+05
Df Residuals:                  295801   BIC:                        -3.552e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1195      0.002    -56.732      0.0

In [11]:
# T1stargazer = Stargazer([T1model1, T1model2, T1model3, T1model4, T1model5, T1model6])
# print(T1stargazer.render_latex())

# Baseline Regressions without Time Fixed Effects (for comparison)

In [12]:
X = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML']])
y = factordata["ExcessReturn"]
model = sm.OLS(y, X).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                 1.209e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:                 92890.
No. Observations:              295829   AIC:                        -1.858e+05
Df Residuals:                  295825   BIC:                        -1.857e+05
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0985      0.000   -303.195      0.0

In [13]:
X = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']])
y = factordata["ExcessReturn"]
model = sm.OLS(y, X).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     7439.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:                 93289.
No. Observations:              295829   AIC:                        -1.866e+05
Df Residuals:                  295823   BIC:                        -1.865e+05
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1004      0.000   -300.251      0.0

In [14]:
X = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_U']])
y = factordata["ExcessReturn"]
model = sm.OLS(y, X).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     9191.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:                 93009.
No. Observations:              295829   AIC:                        -1.860e+05
Df Residuals:                  295824   BIC:                        -1.860e+05
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0981      0.000   -302.332      0.0

In [15]:
X = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_S']])
y = factordata["ExcessReturn"]
model = sm.OLS(y, X).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     9194.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:                 93087.
No. Observations:              295829   AIC:                        -1.862e+05
Df Residuals:                  295824   BIC:                        -1.861e+05
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0990      0.000   -304.627      0.0

In [16]:
X = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U']])
y = factordata["ExcessReturn"]
model = sm.OLS(y, X).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     6222.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:                 93317.
No. Observations:              295829   AIC:                        -1.866e+05
Df Residuals:                  295822   BIC:                        -1.865e+05
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0999      0.000   -294.809      0.0

In [17]:
X = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_S']])
y = factordata["ExcessReturn"]
model = sm.OLS(y, X).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     6224.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:                 93347.
No. Observations:              295829   AIC:                        -1.867e+05
Df Residuals:                  295822   BIC:                        -1.866e+05
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1003      0.000   -300.051      0.0

# Table 2: How does the GMB Factor Change with the Sector?

## Table 2a: GMB_U

**(1) FF3 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects**

In [18]:
T2aX1 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_Oil', 'IndU_RE', 'IndU_Serv', 'IndU_Transp', 'IndU_Util', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2ay1 = factordata["ExcessReturn"]
T2amodel1 = sm.OLS(T2ay1, T2aX1).fit(cov_type='HC1')
print(T2amodel1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                 1.082e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:06   Log-Likelihood:             1.7841e+05
No. Observations:              295829   AIC:                        -3.567e+05
Df Residuals:                  295791   BIC:                        -3.563e+05
Df Model:                          37                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.1282      0.002    -62.784      

**(2) FF3 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects + SECTOR FIXED EFFECTS**

In [19]:
T2aX2 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_Oil', 'IndU_RE', 'IndU_Serv', 'IndU_Transp', 'IndU_Util', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_OilGas', 'sector_RealEstate', 
                          'sector_Services', 'sector_TransportWarehouse', 
                          'sector_Utilities', 'sector_Wholesale']])
T2ay2 = factordata["ExcessReturn"]
T2amodel2 = sm.OLS(T2ay2, T2aX2).fit(cov_type='HC1')
print(T2amodel2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     8206.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:07   Log-Likelihood:             1.7843e+05
No. Observations:              295829   AIC:                        -3.568e+05
Df Residuals:                  295779   BIC:                        -3.562e+05
Df Model:                          49                                         
Covariance Type:                  HC1                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

**(3) FF5 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects**

In [20]:
T2aX3 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_Oil', 'IndU_RE', 'IndU_Serv', 'IndU_Transp', 'IndU_Util', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2ay3 = factordata["ExcessReturn"]
T2amodel3 = sm.OLS(T2ay3, T2aX3).fit(cov_type='HC1')
print(T2amodel3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                 1.032e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:07   Log-Likelihood:             1.7901e+05
No. Observations:              295829   AIC:                        -3.579e+05
Df Residuals:                  295789   BIC:                        -3.575e+05
Df Model:                          39                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.1169      0.002    -55.211      

**(4) FF5 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects + SECTOR FIXED EFFECTS**

In [21]:
T2aX4 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_Oil', 'IndU_RE', 'IndU_Serv', 'IndU_Transp', 'IndU_Util', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_OilGas', 'sector_RealEstate', 
                          'sector_Services', 'sector_TransportWarehouse', 
                          'sector_Utilities', 'sector_Wholesale']])
T2ay4 = factordata["ExcessReturn"]
T2amodel4 = sm.OLS(T2ay4, T2aX4).fit(cov_type='HC1')
print(T2amodel4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     7931.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:08   Log-Likelihood:             1.7903e+05
No. Observations:              295829   AIC:                        -3.580e+05
Df Residuals:                  295777   BIC:                        -3.574e+05
Df Model:                          51                                         
Covariance Type:                  HC1                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [22]:
# T2astargazer = Stargazer([T2amodel1, T2amodel2, T2amodel3, T2amodel4])
# print(T2astargazer.render_latex())

## Table 2b: GMB_S

**(1) FF3 + GMB_S + (Sector Indicators)GMB_S + Time Fixed Effects**

In [23]:
T2bX1 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_Oil', 'IndS_RE', 'IndS_Serv', 'IndS_Transp', 'IndS_Util', 'IndS_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2by1 = factordata["ExcessReturn"]
T2bmodel1 = sm.OLS(T2by1, T2bX1).fit(cov_type='HC1')
print(T2bmodel1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                 1.083e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:09   Log-Likelihood:             1.7808e+05
No. Observations:              295829   AIC:                        -3.561e+05
Df Residuals:                  295791   BIC:                        -3.557e+05
Df Model:                          37                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.1292      0.002    -63.645      

**(2) FF3 + GMB_S + (Sector Indicators)GMB_S + Time Fixed Effects + SECTOR Fixed Effects**

In [24]:
T2bX2 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_Oil', 'IndS_RE', 'IndS_Serv', 'IndS_Transp', 'IndS_Util', 'IndS_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_OilGas', 'sector_RealEstate', 
                          'sector_Services', 'sector_TransportWarehouse', 
                          'sector_Utilities', 'sector_Wholesale']])
T2by2 = factordata["ExcessReturn"]
T2bmodel2 = sm.OLS(T2by2, T2bX2).fit(cov_type='HC1')
print(T2bmodel2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     8218.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:09   Log-Likelihood:             1.7810e+05
No. Observations:              295829   AIC:                        -3.561e+05
Df Residuals:                  295779   BIC:                        -3.556e+05
Df Model:                          49                                         
Covariance Type:                  HC1                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

**(3) FF5 + GMB_S + (Sector Indicators)GMB_S + Time Fixed Effects**

In [25]:
T2bX3 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_Oil', 'IndS_RE', 'IndS_Serv', 'IndS_Transp', 'IndS_Util', 'IndS_Whol',  
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2by3 = factordata["ExcessReturn"]
T2bmodel3 = sm.OLS(T2by3, T2bX3).fit(cov_type='HC1')
print(T2bmodel3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                 1.032e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:10   Log-Likelihood:             1.7873e+05
No. Observations:              295829   AIC:                        -3.574e+05
Df Residuals:                  295789   BIC:                        -3.570e+05
Df Model:                          39                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.1195      0.002    -56.706      

**(4) FF5 + GMB_S + (Sector Indicators)GMB_U + Time Fixed Effects + SECTOR FIXED EFFECTS**

In [26]:
T2bX4 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_Oil', 'IndS_RE', 'IndS_Serv', 'IndS_Transp', 'IndS_Util', 'IndS_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_OilGas', 'sector_RealEstate', 
                          'sector_Services', 'sector_TransportWarehouse', 
                          'sector_Utilities', 'sector_Wholesale']])
T2by4 = factordata["ExcessReturn"]
T2bmodel4 = sm.OLS(T2by4, T2bX4).fit(cov_type='HC1')
print(T2bmodel4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     7932.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:10   Log-Likelihood:             1.7876e+05
No. Observations:              295829   AIC:                        -3.574e+05
Df Residuals:                  295777   BIC:                        -3.569e+05
Df Model:                          51                                         
Covariance Type:                  HC1                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [27]:
# T2bstargazer = Stargazer([T2bmodel1, T2bmodel2, T2bmodel3, T2bmodel4])
# print(T2bstargazer.render_latex())

# Table 3: Estimated vs. Reported Emissions

**FF3 + GMB_U + (Method Indicator)GMB_U + Method Ind.**

In [28]:
T3X1 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'Method_Estimated', 'GMB_U', 'IndGMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y1 = factordata["ExcessReturn"]
T3model1 = sm.OLS(T3y1, T3X1).fit(cov_type='HC1')
print(T3model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                 1.472e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:11   Log-Likelihood:             1.7773e+05
No. Observations:              295829   AIC:                        -3.554e+05
Df Residuals:                  295801   BIC:                        -3.551e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1274      0.002  

**FF5 + GMB_U + (Method Indicator)GMB_U + Method Ind.**

In [29]:
T3X2 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Method_Estimated', 'GMB_U', 'IndGMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y2 = factordata["ExcessReturn"]
T3model2 = sm.OLS(T3y2, T3X2).fit(cov_type='HC1')
print(T3model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                 1.379e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:11   Log-Likelihood:             1.7835e+05
No. Observations:              295829   AIC:                        -3.566e+05
Df Residuals:                  295799   BIC:                        -3.563e+05
Df Model:                          29                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1157      0.002  

**FF3 + GMB_S + (Method Indicator)GMB_S + Method Ind.**

In [30]:
T3X3 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'Method_Estimated', 'GMB_S', 'IndGMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y3 = factordata["ExcessReturn"]
T3model3 = sm.OLS(T3y3, T3X3).fit(cov_type='HC1')
print(T3model3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                 1.472e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:12   Log-Likelihood:             1.7728e+05
No. Observations:              295829   AIC:                        -3.545e+05
Df Residuals:                  295801   BIC:                        -3.542e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1301      0.002  

**FF3 + GMB_S + (Method Indicator)GMB_S + Method Ind.**

In [31]:
T3X4 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Method_Estimated', 'GMB_S', 'IndGMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y4 = factordata["ExcessReturn"]
T3model4 = sm.OLS(T3y4, T3X4).fit(cov_type='HC1')
print(T3model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.377e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:12   Log-Likelihood:             1.7796e+05
No. Observations:              295829   AIC:                        -3.559e+05
Df Residuals:                  295799   BIC:                        -3.555e+05
Df Model:                          29                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1202      0.002  

In [32]:
# T3stargazer = Stargazer([T3model1, T3model2, T3model3, T3model4])
# print(T3stargazer.render_latex())

# Table 4: How Do Other Financial Factors Change Depending on Estimated vs. Reported? ***(Ended up NOT including in report!)***

**FF3 + GMB_U + (Method Indicator)GMB_U + Method Ind. + (Method Indicator)(Fama-French Factors)**

In [33]:
T4X1 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'Method_Estimated', 'GMB_U', 'IndGMB_U',
                                   'IndMkt-RF', 'IndSMB', 'IndHML',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T4y1 = factordata["ExcessReturn"]
T4model1 = sm.OLS(T4y1, T4X1).fit(cov_type='HC1')
print(T4model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.333e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:13   Log-Likelihood:             1.7778e+05
No. Observations:              295829   AIC:                        -3.555e+05
Df Residuals:                  295798   BIC:                        -3.552e+05
Df Model:                          30                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1280      0.002  

**FF5 + GMB_U + (Method Indicator)GMB_U + Method Ind. + (Method Indicator)(Fama-French Factors)**

In [34]:
T4X2 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Method_Estimated', 'GMB_U', 'IndGMB_U',
                                   'IndMkt-RF', 'IndSMB', 'IndHML', 'IndRMW', 'IndCMA',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T4y2 = factordata["ExcessReturn"]
T4model2 = sm.OLS(T4y2, T4X2).fit(cov_type='HC1')
print(T4model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                 1.187e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:13   Log-Likelihood:             1.7841e+05
No. Observations:              295829   AIC:                        -3.568e+05
Df Residuals:                  295794   BIC:                        -3.564e+05
Df Model:                          34                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1164      0.002  

**FF3 + GMB_S + (Method Indicator)GMB_S + Method Ind. + (Method Indicator)(Fama-French Factors)**

In [35]:
T4X3 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'Method_Estimated', 'GMB_S', 'IndGMB_S',
                                   'IndMkt-RF', 'IndSMB', 'IndHML',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T4y3 = factordata["ExcessReturn"]
T4model3 = sm.OLS(T4y3, T4X3).fit(cov_type='HC1')
print(T4model3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                 1.333e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:14   Log-Likelihood:             1.7761e+05
No. Observations:              295829   AIC:                        -3.552e+05
Df Residuals:                  295798   BIC:                        -3.548e+05
Df Model:                          30                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1304      0.002  

**FF5 + GMB_S + (Method Indicator)GMB_S + Method Ind. + (Method Indicator)(Fama-French Factors)**

In [36]:
T4X4 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Method_Estimated', 'GMB_S', 'IndGMB_S',
                                   'IndMkt-RF', 'IndSMB', 'IndHML','IndRMW', 'IndCMA',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T4y4 = factordata["ExcessReturn"]
T4model4 = sm.OLS(T4y4, T4X4).fit(cov_type='HC1')
print(T4model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                 1.186e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:06:14   Log-Likelihood:             1.7834e+05
No. Observations:              295829   AIC:                        -3.566e+05
Df Residuals:                  295794   BIC:                        -3.562e+05
Df Model:                          34                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1201      0.002  